In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler


import tensorflow as tf
from tensorflow import keras
from keras.optimizers import Adam, SGD, Adagrad
from tensorflow.keras import layers
from tensorflow.keras import layers


from sklearn.model_selection import train_test_split

from imblearn.over_sampling import RandomOverSampler

In [2]:
df = pd.read_csv("./diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.shape

(768, 9)

In [4]:
cols = df.columns
cols = list(cols)
cols.remove('Outcome')
X = df[cols]
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [5]:
y = df['Outcome']
y.head()

0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: int64

In [6]:
sm = RandomOverSampler(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X, y)

In [7]:
X_resampled.shape

(1000, 8)

In [8]:
scaler = StandardScaler()

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled.values, y_resampled.values, test_size=0.30, random_state=42)

In [10]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
X_test

array([[-0.3022618 , -0.03963128,  0.52366485, ...,  0.05986596,
        -0.52740821, -0.6814796 ],
       [ 1.14892653, -1.8382816 ,  0.09107551, ..., -0.10287646,
         0.34290932,  0.69998131],
       [ 2.01963952, -0.16157367,  0.52366485, ...,  1.29399604,
         0.88870168,  1.21802915],
       ...,
       [-1.17297479,  0.29571031,  1.28069621, ...,  1.06344427,
        -0.58936302, -0.6814796 ],
       [ 1.72940186, -0.31400167, -3.80222862, ...,  0.34466521,
        -1.03189736, -0.42245568],
       [ 0.85868886,  0.81396549,  0.9562542 , ...,  2.33825995,
        -0.43300089,  0.18193347]])

In [12]:
model = keras.Sequential()
model.add(layers.Dense(500, input_shape=(8,), kernel_initializer='uniform', activation='tanh'))
model.add(layers.Dense(1000, kernel_initializer='uniform', activation='tanh'))
model.add(layers.Dense(500, kernel_initializer='uniform', activation='tanh'))
model.add(layers.Dense(1,kernel_initializer='uniform', activation='sigmoid'))


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 500)               4500      
                                                                 
 dense_1 (Dense)             (None, 1000)              501000    
                                                                 
 dense_2 (Dense)             (None, 500)               500500    
                                                                 
 dense_3 (Dense)             (None, 1)                 501       
                                                                 
Total params: 1,006,501
Trainable params: 1,006,501
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
model.fit(X_train, y_train, epochs=150, batch_size=10,  verbose=2, validation_data=(X_test, y_test))

Epoch 1/150
70/70 - 1s - loss: 0.5686 - accuracy: 0.7186 - val_loss: 0.5019 - val_accuracy: 0.7400 - 916ms/epoch - 13ms/step
Epoch 2/150
70/70 - 0s - loss: 0.5259 - accuracy: 0.7471 - val_loss: 0.5218 - val_accuracy: 0.7433 - 278ms/epoch - 4ms/step
Epoch 3/150
70/70 - 0s - loss: 0.5379 - accuracy: 0.7329 - val_loss: 0.4855 - val_accuracy: 0.7700 - 295ms/epoch - 4ms/step
Epoch 4/150
70/70 - 0s - loss: 0.5356 - accuracy: 0.7357 - val_loss: 0.5215 - val_accuracy: 0.7533 - 301ms/epoch - 4ms/step
Epoch 5/150
70/70 - 0s - loss: 0.5292 - accuracy: 0.7457 - val_loss: 0.4989 - val_accuracy: 0.7400 - 296ms/epoch - 4ms/step
Epoch 6/150
70/70 - 0s - loss: 0.5305 - accuracy: 0.7357 - val_loss: 0.4993 - val_accuracy: 0.7533 - 304ms/epoch - 4ms/step
Epoch 7/150
70/70 - 0s - loss: 0.5312 - accuracy: 0.7514 - val_loss: 0.5055 - val_accuracy: 0.7400 - 288ms/epoch - 4ms/step
Epoch 8/150
70/70 - 0s - loss: 0.5214 - accuracy: 0.7457 - val_loss: 0.5243 - val_accuracy: 0.7333 - 284ms/epoch - 4ms/step
Epoch 9

In [16]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

10/10 [==============================] - 0s 4ms/step - loss: 0.5605 - accuracy: 0.8400
Accuracy: 84.00


In [17]:
predict_x = model.predict(X_test)
classes_x=np.argmax(predict_x,axis=1)

10/10 [==============================] - 0s 3ms/step
